In [1]:
data_var = '2024-01-04'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9041,2024-01-04,Uruguai Liga Uruguaia,22:00,Aguada,Cordon,1.17,4.71,154.5,1.83,1.87,-12.5,2.05,1.69,bZ8kjo3q,0.854701,0.212314,0.546448,0.534759,0.067015,139.318,22.522576,0.161663,2.4,1.341641,0.559017,143.10,1.564,0.239854,0.153359,64.0,197.662,97.504843,0.493291,1.2,1.643168,1.369306,135.20,2.912,1.732057,0.594800,-8.0,90,80,1.59,1.69,126.698,0.000,0.851414,0.015289,0.136128,1.75,0.350,0.485714,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
9042,2024-01-04,Bahrein Premier League,11:30,Bahrain U18,Sitra Club,1.50,2.47,170.5,1.85,1.85,-4.5,1.93,1.78,Gp4hb35d,0.666667,0.404858,0.540541,0.540541,0.071525,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,69,0.00,13.50,0.000,0.000,0.345538,0.000000,0.057178,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
9043,2024-01-04,Eua Ncaa,21:00,Delaware,Hampton,1.08,7.70,152.5,1.80,1.91,-13.5,1.91,1.80,ShZeUcJM,0.925926,0.129870,0.555556,0.523560,0.055796,230.440,226.479575,0.982814,1.8,1.643168,0.912871,214.02,2.806,2.517584,0.897215,17.0,362.780,213.242598,0.587801,0.0,0.000000,NaN,568.75,5.340,3.215082,0.602075,-68.0,82,65,2.61,8.75,0.000,456.340,1.066298,0.041931,0.041931,0.00,0.000,inf,0.0,0.0,0.0,0.15,0.030,223.333333,0.000000,0.0,0.000000
9044,2024-01-04,Eua Ncaa,21:00,Merrimack Warriors,Fairleigh Dickinson,1.31,3.38,146.5,1.83,1.83,-7.5,2.00,1.73,bcHmppa8,0.763359,0.295858,0.546448,0.546448,0.059217,343.468,335.814264,0.977716,1.2,1.643168,1.369306,113.40,5.554,6.049668,1.089245,-11.0,174.608,48.448452,0.277470,1.2,1.643168,1.369306,150.42,2.630,1.273028,0.484041,-47.0,63,69,1.80,2.18,0.000,206.748,0.624184,0.000000,0.102369,0.00,0.000,inf,0.0,0.0,0.0,-1.10,-0.220,-10.818182,0.000000,0.0,0.000000
9045,2024-01-04,Eua Ncaa,21:00,Eastern Kentucky,Lipscomb,2.00,1.80,153.5,1.80,1.91,1.5,1.80,1.91,KUWZtEw3,0.500000,0.555556,0.555556,0.523560,0.055556,432.442,550.050126,1.271963,0.0,0.000000,NaN,1407.00,6.154,8.333737,1.354198,-74.0,245.692,101.318656,0.412381,1.2,1.643168,1.369306,344.76,3.608,1.600787,0.443677,-9.0,67,78,21.00,4.42,0.000,248.970,0.074432,0.041931,0.041931,0.00,0.000,inf,0.0,0.0,0.0,0.87,0.174,4.597701,0.393935,0.4,0.006065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9139,2024-01-04,Eua Ncaa,21:00,Michigan State,Penn State,1.10,7.20,145.5,1.80,1.91,-14.5,1.91,1.80,Oni0qzB6,0.909091,0.138889,0.555556,0.523560,0.047980,119.336,42.063996,0.352484,2.4,1.341641,0.559017,107.01,1.422,0.488743,0.343701,90.0,166.746,60.391399,0.362176,1.8,1.643168,0.912871,97.20,2.112,0.829108,0.392570,40.0,87,90,1.23,1.08,114.186,0.000,1.039362,0.041931,0.041931,0.48,0.096,1.041667,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
9140,2024-01-04,Eua Ncaa,21:00,Hofstra,Coll Of Charltn,1.73,2.10,153.5,1.83,1.83,-3.5,1.91,1.80,d8PROysp,0.578035,0.476190,0.546448,0.546448,0.054225,270.624,244.508456,0.903499,0.6,1.341641,2.236068,349.18,3.904,3.522645,0.902317,-31.0,226.100,259.789256,1.149002,2.4,1.341641,0.559017,157.53,2.884,3.598254,1.247661,24.0,79,89,4.42,

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
9045,21:00,Eua Ncaa,Eastern Kentucky,Lipscomb,2.00,Back Home
9049,21:00,Eua Ncaa,Detroit,Wisc. Green Bay,2.23,Back Home
9055,15:04,Suécia Basketligan,Sodertalje,Jamtland,1.98,Back Home
9058,14:00,Sérvia Primeira Liga,OKK Novi Pazar,Sloboda,2.39,Back Home
9060,13:30,Polônia Liga De Basquete,Torun,Lublin,1.74,Back Home
9061,21:00,Eua Ncaa,Monmouth (N.J.),Towson,2.18,Back Home
9062,21:00,Uruguai Liga Uruguaia,Malvin,Nacional,1.76,Back Home
9065,21:00,Eua Ncaa,Northern Kentucky,Youngstown State,1.83,Back Home
9072,22:30,Eua Ncaa,Southern Indiana,Tennessee State,2.35,Back Home
9073,22:00,Eua Ncaa,Utah Valley,California Baptist,1.73,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
9107,16:15,Europa Euroliga,Olympiakos,Monaco,1.95,Back Home
9109,16:00,Europa Euroliga,Crvena zvezda,Partizan,1.91,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9077,22:00,Eua Ncaa,Southern Miss,Georgia State,1.64,Back Home
9120,13:00,Lituânia Nkl,Silute,Vilniaus Stekas,1.74,Back Home
9141,21:00,Eua Ncaa,Drexel,NC Wilmington,1.69,Back Home
